In [70]:
import os
from dotenv import load_dotenv

load_dotenv()


api_key=os.getenv("GEMINI_API_KEY")


### 1. initialize Model

In [71]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

llm = ChatGoogleGenerativeAI(
    api_key = api_key,
    model="gemini-2.0-flash"
)

In [72]:
llm.invoke("tell me a joke")

AIMessage(content="Why don't scientists trust atoms?\n\nBecause they make up everything!", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-fea82a78-7ea6-4425-b97f-72a101d78cbe-0', usage_metadata={'input_tokens': 4, 'output_tokens': 16, 'total_tokens': 20, 'input_token_details': {'cache_read': 0}})

In [ ]:
import re
import os
from datetime import datetime
from datetime import timedelta
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, AIMessage
from langchain.chains.conversation.memory import ConversationBufferMemory
import dateparser


llm = ChatGoogleGenerativeAI(
    api_key = api_key,
    model="gemini-2.0-flash"
)

# Define the system message template
template = """
You are a helpful assistant. Your duty is to educate people about who Zabihullah is and help them schedule an appointment.

**Important Rules:**
- Do not generate user responses on your own and avoid repeating questions.
- At the start of every conversation, greet the user with: 
  "Hi, This is AI Agent, How may I help you today?"
- You can only book an appointment for Zabihullah, who offers:
  - Solution consulting
  - Full-stack development
  - AI/ML development
- Working hours: **8 AM - 10 PM **
- Ask for and collect these details:
  - Full Name
  - Service Type
  - Location
  - Date & Time (Accept any format and convert to  12-hour format)
  - Email Address
- Confirm appointment details at the end.
- Respond with 'Thank you for connecting' after booking.

**Chat History:**
{chat_history}
"""

# Create the prompt template
prompt_template = ChatPromptTemplate.from_messages([
    ("system", template),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

# Initialize conversation memory properly
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Create the chain using the updated LangChain pattern to avoid deprecation warning
chain = prompt_template | llm

# Function to extract information using regular expressions
def extract_information(text, pattern):
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        return match.group(1).strip()
    return None

# Function to convert user-provided time to IST 24-hour format
def convert_to_ist(user_time_str):
    try:
        # Use dateparser for flexible date parsing
        user_time = dateparser.parse(user_time_str)
        if not user_time:
            return None
            
        # Check if time zone info exists, if not assume UTC
        if user_time.tzinfo is None:
            # Add UTC timezone
            user_time = user_time.replace(tzinfo=datetime.timezone.utc)
            
        # Convert to IST (UTC+5:30)
        ist_offset = timedelta(hours=5, minutes=30)
        ist_time = user_time + ist_offset
        return ist_time.strftime("%Y-%m-%d %H:%M")
    except Exception:
        return None

def chatbot_conversation():
    # Dictionary to store user responses
    user_data = {
        "Full Name": None,
        "Service Type": None,
        "Location": None,
        "Date & Time": None,
        "Email Address": None
    }
    
    # Initial greeting
    initial_greeting = "Hi, This is AI Agent, How may I help you today?"
    print(f"AI: {initial_greeting}")
    
    # Store the messages we want to track
    messages = []
    
    # Main conversation loop
    conversation_active = True
    appointment_confirmed = False
    
    while conversation_active:
        # Get user input
        user_input = input("You: ")
        
        # Check if we need to exit
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("AI: Thank you for connecting.")
            break
        
        # Add user message to history
        messages.append(HumanMessage(content=user_input))
        
        # Process with LLM using the new pattern
        chat_history = messages.copy()
        ai_response = chain.invoke({"input": user_input, "chat_history": chat_history})
        
        # Get the response text
        ai_response_text = ai_response.content
        print(f"AI: {ai_response_text}")
        
        # Add AI response to history
        messages.append(AIMessage(content=ai_response_text))
        
        # Extract information based on patterns
        if not user_data["Full Name"] and re.search(r'name', ai_response_text, re.IGNORECASE):
            user_data["Full Name"] = extract_information(user_input, r'([\w\s]+)')
            
        if not user_data["Service Type"] and re.search(r'service', ai_response_text, re.IGNORECASE):
            service_types = ["solution consulting", "full-stack development", "ai/ml development"]
            for service in service_types:
                if service.lower() in user_input.lower():
                    user_data["Service Type"] = service
                    break
                    
        if not user_data["Location"] and re.search(r'location', ai_response_text, re.IGNORECASE):
            user_data["Location"] = extract_information(user_input, r'([\w\s,]+)')
            
        if not user_data["Date & Time"] and re.search(r'date|time', ai_response_text, re.IGNORECASE):
            ist_time = convert_to_ist(user_input)
            if ist_time:
                user_data["Date & Time"] = ist_time
                
        if not user_data["Email Address"] and re.search(r'email', ai_response_text, re.IGNORECASE):
            email_pattern = r'([a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,})'
            user_data["Email Address"] = extract_information(user_input, email_pattern)
        
        # Check if all data is collected and confirmation needed
        if all(user_data.values()) and not appointment_confirmed:
            confirmation_message = f"""
####

Dear {user_data['Full Name']},

I hope this email finds you well. We are writing to confirm the details of your scheduled quick call with Zabihullah.
Please review the information below:

Service Type: {user_data['Service Type']}
Location: {user_data['Location']}
Date & Time (IST): {user_data['Date & Time']}
Link: [Google Meet Link]

If you have any questions or need to make any changes, please don't hesitate to reach out to us. You can reply to this email or contact us directly at (000)-000 0000.

We look forward to meeting with you on {user_data['Date & Time']}. Thank you for choosing our service, and we are excited to assist you.

Best regards,
AI Agent

####
"""
            print(f"AI: {confirmation_message}")
            messages.append(AIMessage(content=confirmation_message))
            appointment_confirmed = True
            print("AI: Thank you for connecting.")
            conversation_active = False

# Run the chatbot conversation
if __name__ == "__main__":
    try:
        chatbot_conversation()
    except KeyboardInterrupt:
        print("\nConversation ended by user.")
    except Exception as e:
        print(f"\nAn error occurred: {str(e)}")

AI: Hi, This is AI Agent, How may I help you today?
AI: Hi, This is AI Agent, How may I help you today?
AI: Okay, I can help you with that. To schedule an appointment, I need a few details.

Could you please provide the following information?

*   **Full Name:**
*   **Service Type:** (Solution consulting, Full-stack development, or AI/ML development)
*   **Location:**
*   **Date & Time:** (Please specify your preferred date and time. I will convert it to IST)
*   **Email Address:**
AI: Okay, I have the following information:

*   **Full Name:** zabihullah
*   **Service Type:** AI/ML development
*   **Location:** peshawar
*   **Date & Time:** 2025-03-13 09:30 (Converting to IST)
*   **Email Address:** zabihullah18381@gmail.com

Just to confirm, the appointment is for **zabihullah** for **AI/ML development** in **peshawar** on **March 13, 2025, at 09:30 Peshawar time** (which will be converted to IST), and the email address is **zabihullah18381@gmail.com**. Is that correct?
AI: Excellent